# MilvusDB 사용하기
milvusdb에 콜렉션을 생성하고 스키마를 정의한 후 데이터를 입력합니다.
입력한 데이터를 검색합니다.
작업이 완료되면 생성한 컬렉션을 삭제합니다.

In [ ]:
!pip install chromadb==0.4.2
!pip install langchain==0.3.0
!pip install langchain --upgrade
!pip install flask-sqlalchemy --user
!pip install pypdf 
!pip install sentence-transformers
!pip install langchain_openai
!pip install langchain_community

!pip install ibm-cos-sdk==2.12.0
!pip install ibm-cos-sdk-core==2.12.0
!pip install ibm-cos-sdk-s3transfer==2.12.0
!pip install ibm-watson-machine-learning==1.0.356
!pip install pymilvus
!pip install -U langchain-ibm

라이브러리 설치가 완료 되었으면 노트북 커널을 Restart 합니다.
메뉴에서 `kernel > restart` 를 실행합니다.

In [ ]:
# pip show pymilvus --files

### Milvusdb  접속
Milvus 클라이언트 설치

In [57]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections,utility,Collection,CollectionSchema, FieldSchema,DataType
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.docstore.document import Document


milvus 접속 정보를 변경합니다.   

In [58]:
MILVUS_DB_HOST = "<MILVUS_DB_HOST>"
MILVUS_DB_PORT = "19530"


컬렉션 명을 지정합니다.   
규칙 : kr_covid19_랩번호

In [59]:
COLLECTION_NAME="kr_genai_lab_<YOUR NAME>"
PDF_FILE_NAME='KR-TMT2024-GEN-AI.pdf'

## Using Langchain

In [ ]:
# Import library
from ibm_watson_studio_lib import access_project_or_space
from langchain.chains import RetrievalQA

IBM Cloud의 watsonx 서비스에서 프로젝트 토큰 정보를 가져옵니다.   
메뉴는 `Project > Manage > Access Contorl > Access Tokens`입니다. 생성된 토큰 정보를 복사하여 `<YOUR ACCESS TOKEN HERE>`값을 삭제하고 여기에 붙여넣습니다.   

In [ ]:
# Create access token in project
token = "<YOUR ACCESS TOKEN HERE>"
wslib = access_project_or_space({"token":token})
wslib.download_file(PDF_FILE_NAME)

In [60]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


loader = PyPDFLoader(PDF_FILE_NAME)
pages = loader.load_and_split()

In [61]:
len(pages)

26

청크는 기본적으로 문자수를 가지고 길이를 측정합니다. chunk_size는 분할될 최대 문자수입니다.
* 텍스트 분할 방법: 단일 문자 기준.
* 청크 크기 측정 방법: 문자 수 기준.

In [62]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=10000, 
    chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [63]:
len(docs)


26

청크 내용을 확인합니다.

In [64]:
i=0
for doc in docs:
    i+=1
    print(">청크 번호 : ",i)
    print(">청크 사이즈 : ",len(docs[i-1].page_content))
    print(">청크 내용 : ")
    print(doc.page_content)
    print("\n")

>청크 번호 :  1
>청크 사이즈 :  69
>청크 내용 : 
코로나바이러스감염증 (COVID)-19 
검역대응 지침(제14-3판)
2023.  6. 1.
중앙방역대책본부
해외출입국관리팀


>청크 번호 :  2
>청크 사이즈 :  130
>청크 내용 : 
본 대응 지침은 코로나바이러스감염증 -19 국내 유입에 따라「코로나바이러스감염증 -19 
대응지침 」을 근거로 국립검역소 검역대응을 위해 마련되었습니다 . 향후 감염병 
위기경보 수준 및 발생 상황에 따라 보완·변경될 수 있습니다 .


>청크 번호 :  3
>청크 사이즈 :  2774
>청크 내용 : 
목   차
Ⅰ. 대응원칙 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  1
Ⅱ. 사례정의 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  2
Ⅲ. 공항 검역대응 절차· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  4
       1. 공항 검역대응 주요사항 안내························································································  4
       2. 검역대응 흐름도 ······

임베딩 모델을 지정합니다. 한글 지원이 되는 ```distiluse-base-multilingual-cased-v1```를 지정합니다.

In [65]:
# embedding_model_name = "distiluse-base-multilingual-cased-v1"
embedding_model_name = "intfloat/multilingual-e5-base"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [66]:
len(docs)

26

In [67]:
print(docs)

[Document(page_content='코로나바이러스감염증 (COVID)-19 \n검역대응 지침(제14-3판)\n2023.  6. 1.\n중앙방역대책본부\n해외출입국관리팀', metadata={'source': './sample_data/Korea_sample_pdf_refined.pdf', 'page': 0}), Document(page_content='본 대응 지침은 코로나바이러스감염증 -19 국내 유입에 따라「코로나바이러스감염증 -19 \n대응지침 」을 근거로 국립검역소 검역대응을 위해 마련되었습니다 . 향후 감염병 \n위기경보 수준 및 발생 상황에 따라 보완·변경될 수 있습니다 .', metadata={'source': './sample_data/Korea_sample_pdf_refined.pdf', 'page': 1}), Document(page_content='목   차\nⅠ. 대응원칙 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  1\nⅡ. 사례정의 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  2\nⅢ. 공항 검역대응 절차· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  4\n       1. 공항 검역

랭체인 Milvus를 객체를 이용하여 문서를 벡터 디비에 인서트 합니다.  

In [68]:
vector_db_milvus = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": MILVUS_DB_HOST, "port": MILVUS_DB_PORT},
    collection_name = COLLECTION_NAME, ## custom collection name 
    search_params = {"metric":"IP","offset":0}, ## search params
)

옵션) 저장된 정보를 조회하기 위해 사용

In [54]:
# vector_db_milvus = Milvus(
#             embeddings,
#             connection_args={"host": MILVUS_DB_HOST, "port": MILVUS_DB_PORT},
#             collection_name = COLLECTION_NAME, ## custom collection name 
#             search_params = {"metric":"IP","offset":0}, ## search params
# )

Milvus DB에 저장된 데이터를 조회 합니다.

In [69]:
query = "기업의 생성형AI 도입 현황은?"
docs = vector_db_milvus.similarity_search(
    query,
    k=2
    )

In [70]:
len(docs)

2

In [71]:
for doc in docs:
    print(doc)
    print()

page_content='코로나바이러스감염증 (COVID)-19 \n검역대응 지침(제14-3판)\n2023.  6. 1.\n중앙방역대책본부\n해외출입국관리팀' metadata={'source': './sample_data/Korea_sample_pdf_refined.pdf', 'page': 0}

page_content='- 2 -II. 사례정의 등\n 본 사례정의는 국내 유입된 코로나 19의 위기경보 수준 ‘경계단계 ’ 상황에 적용하며 , \n「코로나바이러스 19 대응 지침(지자체용 )」변경에 따라 사례정의가 일부 변경될 수 있음\n 1. 사례정의\n▶ 진단을 위한 검사 기준 : 코로나 19 유전자 (PCR) 검출, 바이러스 분리\n  - 신속항원검사 (전문가용 ) 또는 응급용 선별검사 결과 양성*\n    * 한시 시행 (’22. 3. 14. ~ 별도 안내시까지 )\n▶ 주요 임상증상 : 발열(37.5℃ 이상), 기침, 호흡곤란 , 오한, 근육통 , 두통, 인후통 , 후각·\n미각소실 또는 폐렴 등\n○확진환자\n-코로나 19진단을위한검사기준에따라감염이확인된사람\n-코로나 19증상을나타내는사람으로서신속항원검사 (전문가용 )또는\n응급용선별검사결과양성으로확인되어의사가진단한사람*\n▶ 한시 시행 (’22. 3. 14. ~ 별도 안내시까지 )\n▶ 증상이 있고, 신속항원검사 (전문가용 ) 양성 시 PCR 검사는 미시행 원칙, 다만 의사 \n판단하에  추가 PCR 진행 가능\n ※ 주의: 신속항원검사 (전문가용 ), 응급용 선별검사 결과 양성이더라도 추가 PCR 검사를  \n의뢰하는 경우 확진 환자로 신고하지 않고 PCR 검사 양성 시 신고\n※ 검역단계 사례분류는 사례정의를 참고하여 역학조사관 또는 공중보건의가 \n판단할 수 있음' metadata={'source': './sample_data/Korea_sample_pdf_refined.pdf', 'page': 5}



# MilvusDB를 적용한 RAG 시스템 구현
MilvusDB는 벡터 검색 및 분석에 특화된 NoSQL 데이터베이스입니다. 텍스트, 이미지, 오디오 등 다양한 데이터를 벡터 형태로 저장하고 효율적인 검색을 제공합니다.   
1. 문서 벡터화: MilvusDB에 저장할 문서를 벡터 형태로 변환합니다. 텍스트 문서의 경우 TF-IDF, Word2Vec, BERT 등 다양한 방법을 사용하여 벡터화할 수 있습니다.
2. MilvusDB에 데이터 저장: 벡터화된 문서를 MilvusDB에 저장합니다. MilvusDB는 벡터 검색 속도가 빠르기 때문에 질의와 관련된 문서를 빠르게 찾을 수 있습니다.
3. 질의 벡터화: 사용자 질의를 벡터 형태로 변환합니다. 질의 벡터화 방법은 문서 벡터화 방법과 동일할 수 있습니다.
4. MilvusDB에서 유사 문서 검색: 질의 벡터를 기반으로 MilvusDB에서 유사한 문서를 검색합니다. 검색된 문서는 RAG 시스템에서 새로운 텍스트를 생성하는 데 사용됩니다.
새로운 텍스트 생성: 검색된 문서를 기반으로 새로운 텍스트를 생성합니다. Transformer 기반 모델, Seq2Seq 모델 등 다양한 모델을 사용하여 새로운 텍스트를 생성할 수 있습니다.

In [ ]:
from pymilvus import (
    connections,
    Collection
)
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain

from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [ ]:
def milvus_search(query, COLLECTION_NAME,embedding_model_name):
    model_name = embedding_model_name
    collection_name = COLLECTION_NAME
    connections.connect("default", host=MILVUS_DB_HOST, port=MILVUS_DB_PORT) 
    collection = Collection(collection_name)
    collection.load()
    search_params = {
        "metric_type": "L2",
        "params": {"ef": 10},
    }
    model = SentenceTransformer(model_name)
    vectors_to_search = model.encode([query]).tolist()

    result = collection.search(vectors_to_search, "vector", search_params,
                                limit=3,
                                output_fields=["text", "vector"],
                                )

    hits = result[0]
    def text2doc(t):
        return Document(page_content = t)

    docs = [text2doc(h.entity.get('text')) for h in hits]
    return docs

In [ ]:
api_key = "<CLOUD_API_KEY>"
# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id = "<MY_PROJECT_ID>"

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [ ]:
# watsonx model 초기화
params = {
#     GenParams.DECODING_METHOD: "sample",
#     GenParams.TEMPERATURE: 0.2,
#     GenParams.TOP_P: 1,
#     GenParams.TOP_K: 100,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.TEMPERATURE:  0.7,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 1000
}

model_llm = Model(
    model_id="meta-llama/llama-3-70b-instruct",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "프라이빗 모델을 가치는?"
# user_input = "생성형AI 운용 시 비용증가는?"
docs_search = milvus_search(user_input,COLLECTION_NAME,embedding_model_name)
print(docs_search)

In [ ]:
chain_types = "stuff"
# chain_types = "map_reduce"
# chain_types = "refine"

chain = load_qa_chain(model_llm, chain_type=chain_types)
response = chain.run(input_documents=docs_search, question=user_input+". 한국어로 답하시오.")
print(response)

# 자원삭제

핸즈온을 완료후 자신이 생성한 콜렉션을 삭제합니다.

In [72]:
from pymilvus import (
    connections,
    Collection
)
connections.connect(
  alias="default",
  host=MILVUS_DB_HOST, # YOUR IP
  port=MILVUS_DB_PORT      # YOUR PORT
)

utility.drop_collection(COLLECTION_NAME)

connections.disconnect("default")